In [ ]:
# for google colab
!pip install openap openap-top fastmeteo

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import openap
import pandas as pd
from fastmeteo.source import ArcoEra5
from openap import top


## define the aircraft type, origin and destination airports

In [ ]:
typecode = "A320"
origin = "EHAM"
destination = "LGAV"

# initial mass as the faction of maximum takeoff mass
m0 = 0.85

## simple fuel optimal trajectory

In [ ]:
optimizer = top.CompleteFlight(typecode, origin, destination, m0=m0)

flight = optimizer.trajectory(objective="fuel")

flight.head()

In [ ]:
top.vis.trajectory(flight, windfield=None, barb_steps=15)

## enable the wind

In [ ]:
# get the boundary of the wind field
o = openap.nav.airport(origin)
d = openap.nav.airport(destination)

latmin = round(min(o["lat"], d["lat"])) - 2
latmax = round(max(o["lat"], d["lat"])) + 2
lonmin = round(min(o["lon"], d["lon"])) - 4
lonmax = round(max(o["lon"], d["lon"])) + 4

# create the and flatten the wind grid
latitudes = np.arange(latmin, latmax, 0.5)
longitudes = np.arange(lonmin, lonmax, 1)
altitudes = np.arange(1000, 46000, 2000)
timestamps = pd.date_range("2022-02-20 08:00:00", "2022-02-20 11:00:00", freq="1H")

latitudes, longitudes, altitudes, times = np.meshgrid(
    latitudes, longitudes, altitudes, timestamps
)

grid = pd.DataFrame().assign(
    latitude=latitudes.flatten(),
    longitude=longitudes.flatten(),
    altitude=altitudes.flatten(),
    timestamp=times.flatten(),
)

# obtain the wind based on the grid
era5_grid = ArcoEra5(local_store="/tmp/era5-zarr")

meteo_data = era5_grid.interpolate(grid)

In [ ]:
meteo_data.head()

In [ ]:
wind = (
    meteo_data.rename(
        columns={
            "u_component_of_wind": "u",
            "v_component_of_wind": "v",
        }
    )
    .assign(ts=lambda x: (x.timestamp - x.timestamp.iloc[0]).dt.total_seconds())
    .eval("h=altitude * 0.3048")
)[["ts", "latitude", "longitude", "h", "u", "v"]]

wind.head()

In [ ]:
optimizer = top.CompleteFlight(typecode, origin, destination, m0)
optimizer.enable_wind(wind)
flight = optimizer.trajectory(objective="fuel")

In [ ]:
top.vis.trajectory(flight, windfield=wind, barb_steps=15)

## Using a imaginary cost grid over Europe

In [ ]:
def gaussian(x, y, z, t):
    return (
        np.exp(-((x + 3 - i) ** 2 + (y - 1) ** 2))
        + np.exp(-((x - 3 + i / 2) ** 2 + (y + 2) ** 2))
    ) * z**2


x = np.linspace(-8, 8, 40)
y = np.linspace(-4, 4, 40)
z = np.linspace(1, 2, 32)
X, Y, Z = np.meshgrid(x, y, z)

# add the time dimension
ts = np.arange(0, 8 * 1800, 1800)  # every 30 minutes
costs4d = np.zeros((len(x), len(y), len(z), len(ts)))

for i, ts_ in enumerate(ts):
    costs3d = gaussian(X, Y, Z, ts)
    costs4d[:, :, :, i] = costs3d

costs4d = (costs4d - costs4d.min()) / (costs4d.max() - costs4d.min())

# scale the x,y to lon,lat bound
lon = np.interp(x, (min(x), max(x)), (-10, 40))
lat = np.interp(y, (min(y), max(y)), (35, 60))
alt = np.interp(z, (min(z), max(z)), (0, 40_000))

lons, lats, alts, tss = np.meshgrid(lon, lat, alt, ts)

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(16, 18), subplot_kw={"projection": "3d"})
for i, ax in enumerate(axes.flatten()):
    ax.plot_surface(
        lons[:, :, i // 4 * 6, i % 4 * 2],
        lats[:, :, i // 4 * 6, i % 4 * 2],
        costs4d[:, :, i // 4 * 6, i % 4 * 2],
        edgecolor="tab:blue",
        lw=0.5,
        alpha=0.3,
    )
    ax.set_zlim(0, 1)
    flight_level = int(alt[i // 4 * 6] // 1000 * 10)
    time = ts[i % 4 * 2] / 1800 / 2
    ax.set_title(f"FL{flight_level} | {int(time)}h")
    ax.set_xlabel("longitude")
    ax.set_ylabel("latitude")
    ax.set_zlabel("cost", rotation=90)

plt.show()

In [ ]:
df_cost_4d = pd.DataFrame(
    np.array([lons, lats, alts, tss, costs4d]).reshape(5, -1).T,
    columns=["longitude", "latitude", "altitude", "ts", "cost"],
).assign(height=lambda x: x.altitude * 0.3048)

df_cost_4d

In [ ]:
# optimizer = top.CompleteFlight(typecode, origin, destination, m0=m0)

optimizer = top.Cruise(typecode, origin, destination, m0=m0)

# optimizer.setup(debug=True)

interpolant = top.tools.interpolant_from_dataframe(df_cost_4d)


def objective(x, u, dt, **kwargs):
    """The final objective is the compound of grid cost and fuel"""
    grid_cost = optimizer.obj_grid_cost(
        x, u, dt, n_dim=4, time_dependent=True, **kwargs
    )
    fuel_cost = optimizer.obj_fuel(x, u, dt, **kwargs)
    return grid_cost + fuel_cost * 2


# generate the flight trajectory
# interpolant is passed to trajectory(), and internally used by obj_grid()
flight = optimizer.trajectory(objective=objective, interpolant=interpolant)

In [ ]:
top.vis.trajectory(flight)

In [ ]:
def flight_level_cost_4d(flight, df_cost):
    from cartopy import crs as ccrs
    from cartopy.feature import BORDERS

    proj = ccrs.PlateCarree()

    fig, axes = plt.subplots(
        3,
        2,
        figsize=(9, 9),
        subplot_kw=dict(
            projection=ccrs.TransverseMercator(
                central_longitude=15, central_latitude=45
            )
        ),
    )

    for i, ax in enumerate(axes.flatten()):
        ax.set_extent([-10, 40, 32, 60])
        ax.add_feature(BORDERS, lw=0.5, color="gray")
        ax.coastlines(resolution="110m", lw=0.5, color="gray")

        df_cost_pivot = df_cost.query(
            f"height=={df_cost.height.max()} and ts=={i * 1800}"
        ).pivot(index="latitude", columns="longitude", values="cost")

        lat, lon, val = (
            df_cost_pivot.index.values,
            df_cost_pivot.columns.values,
            df_cost_pivot.values,
        )

        ax.contourf(lon, lat, val, transform=proj, alpha=0.7, cmap="Purples")

        current = flight.query(f"{i * 1800}<ts<{i * 1800 + 600}").iloc[0]

        ax.text(
            0.03, 0.9, f"Time={int(current.ts)}s", transform=ax.transAxes, fontsize=14
        )

        ax.scatter(current.longitude, current.latitude, color="r", lw=5, transform=proj)

        ax.plot(flight.longitude, flight.latitude, color="k", lw=1, transform=proj)

        for r, p in flight.iloc[[0, -1]].iterrows():
            ax.scatter(p.longitude, p.latitude, c="k", transform=proj)

    plt.tight_layout()
    plt.show()


flight_level_cost_4d(flight, df_cost_4d)